# Vamos aprender Deep Learning

In [ ]:
%matplotlib inline
import pandas
import numpy
import matplotlib.pyplot as matplotlib
import seaborn
import string

from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score
from sklearn.model_selection import train_test_split

from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical

from sklearn.neural_network import MLPClassifier

In [ ]:
# Vamos ler o csv de Linguagem de sinais ('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')

Sinais = pandas.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
Sinais

In [ ]:
# E agora vamos criar labels para cada letra

labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

# ou também, da biblioteca string
labels = list(string.ascii_uppercase)
labels

In [ ]:
# Agora, vamos ver o total número de imagens, pegar os x (ou seja, tirar o label) e transformar todas as linhas em matriz de 28x28c
num_images = len(Sinais.index)
imagens = numpy.array(Sinais.drop(['label'], axis=1))
imagens = imagens.reshape(num_images,28,28)

In [ ]:
matplotlib.figure(figsize=(10,20))
for i in range(0,49) :
    matplotlib.subplot(10,5,i+1)
    matplotlib.axis('off')
    matplotlib.imshow(imagens[i], cmap="gray_r")
    matplotlib.title(labels[Sinais.label[i]])

In [ ]:
# Pegar x e y, normalizar os valores de x de 0 e 255 para entre 0 e 1
y = Sinais['label']
x = Sinais.drop(['label'] , axis=1)
x = x/255

# Fazer o train test split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [ ]:
# Vamos começar utilizando o MLPClassifier (Multi-Layer Perceptron) do Sklearn
# Vamos criar um modelo perceptron com duas camadas de 200 e 60 neurônios

MLP = MLPClassifier(hidden_layer_sizes=(200,60))
MLP.fit(x_train,y_train)
predicted = MLP.predict(x_test)
print(accuracy_score(predicted, y_test))

In [ ]:
# Vamos ver a matriz de confusão de A a Z

pandas.crosstab(y_test, predicted, rownames=['Reel'], colnames=['Prediction'], margins=True)

In [ ]:
# Antes de montar nosso próprio modelo, precisamos pegar quantas classes temos no total.
# Você pode ver que a matriz anterior é 25x25, mas para ter certeza, vale a pena pegarmos y e contarmos.
from keras.utils.np_utils import to_categorical

print(y[0])
y_cat = to_categorical(y)
print(y_cat[0])
num_classes = y_cat.shape[1]
print(num_classes)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Antes, o Keras exige que ao invés de dataset eles sejam numpy arrays. Vamos reseparar x e y e usar o numpy.array
x_train, x_test, y_train, y_test = train_test_split(x, y_cat, test_size=0.2, random_state=1)
x_train = numpy.array(x_train)
x_test = numpy.array(x_test)
y_train = numpy.array(y_train)
y_test = numpy.array(y_test)


# Primeiro, puxe o modelo sequencial (cujo os dados vão passar "em sequência", da forma tradicional)
# E vamos fazer duas camadas densas ReLU (Rectified Linear Unit) e uma última camada softmax que vai nos inidicar qual tipo de classe é


model = Sequential()
model.add(Dense(200, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# E agora vamos compilar nosso modelo utilizando a métrica de accuracy e a perda como crossentropy (categorical = classes, crossentropy = "distância" entre elas)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# E agora, salvamos em treinamento o nosso fit de modelo, utilizando os x e y's que separamos. Vamos fazer 30 épocas e colocar a opção verbosa como 1
# Ele já fara o "predict" quando colocarmos validation_data
treinamento = model.fit(x_train , y_train , validation_data=(x_test,y_test), epochs=30, verbose=1)

In [ ]:
# E agora, vamos avaliar
model.evaluate(x_test,y_test)

In [ ]:
# A varíavel treinamento tem salva a accuracy e validação de cada treinamento

print(treinamento.history['accuracy'])
print(treinamento.history['val_accuracy'])

In [ ]:
# E agora, vamos imprimir 
def plot_scores(train) :
    accuracy = train.history['accuracy']
    val_accuracy = train.history['val_accuracy']
    epochs = range(len(accuracy))
    matplotlib.plot(epochs, accuracy, 'b', label='Score apprentissage')
    matplotlib.plot(epochs, val_accuracy, 'r', label='Score validation')
    matplotlib.title('Scores')
    matplotlib.legend()
    matplotlib.show()
plot_scores(treinamento)

E agora você pode repetir os passos para o FashionMNIST

In [ ]:
Roupas = pandas.read_csv('../input/fashionmnist/fashion-mnist_train.csv')
labels = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]
num_images = len(Roupas.index)
imagens = numpy.array(Roupas.drop(['label'], axis=1))
imagens = imagens.reshape(num_images,28,28)
matplotlib.figure(figsize=(10,20))
for i in range(0,49) :
    matplotlib.subplot(10,5,i+1)
    matplotlib.axis('off')
    matplotlib.imshow(imagens[i], cmap="gray_r")
    matplotlib.title(labels[Roupas.label[i]])